In [17]:
from icecream import ic

import torch

print(torch.version.cuda)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)
ic(torch.cuda.is_available())

def is_opencl_available():
    # Check if PyTorch has been built with OpenCL support
    return torch.cuda.is_available() and 'opencl' in torch.cuda.get_device_capability()

# Check if OpenCL is available
if is_opencl_available():
    print("OpenCL is available.")
else:
    print("OpenCL is not available.")


ic| torch.cuda.is_available(): False


11.8
cpu
OpenCL is not available.


In [6]:
with open('pg420.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Checking file contents
ic(len(text))
ic(text[:200])
chars = sorted(set(text))
ic(chars)
ic(len(chars))
vocabulary_size = len(chars)

ic| len(text): 230469
ic| text[:200]: ('\ufeff
                '
                 '
                '
                 'Dorothy and the Wizard in Oz
                '
                 '
                '
                 '
                '
                 '  A Faithful Record of Their Amazing Adventures
                '
                 '    in an Underground World; and How with the
                '
                 '     Aid of Their Friends Zeb Hugson, Eureka
                '
                 '       the Kitten, and Jim ')
ic| chars: ['
           ',
            ' ',
            '!',
            '"',
            '&',
            "'",
            '(',
            ')',
            ',',
            '-',
            '.',
            '0',
            '1',
            '2',
            '3',
            '4',
            '5',
            '6',
            '7',
            '8',
            '9',
            ':',
            ';',
            '?',
            'A',
            'B',
           

In [11]:
# Tokenizer

string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

ic(encode('hello'))
encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)
ic(decoded_hello)

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

ic| encode('hello'): [56, 53, 60, 60, 63]
ic| decoded_hello: 'hello'


tensor([75,  0,  0, 27, 63, 66, 63, 68, 56, 73,  1, 49, 62, 52,  1, 68, 56, 53,
         1, 46, 57, 74, 49, 66, 52,  1, 57, 62,  1, 38, 74,  0,  0,  0,  1,  1,
        24,  1, 29, 49, 57, 68, 56, 54, 69, 60,  1, 41, 53, 51, 63, 66, 52,  1,
        63, 54,  1, 43, 56, 53, 57, 66,  1, 24, 61, 49, 74, 57, 62, 55,  1, 24,
        52, 70, 53, 62, 68, 69, 66, 53, 67,  0,  1,  1,  1,  1, 57, 62,  1, 49,
        62,  1, 44, 62, 52, 53, 66, 55, 66, 63])


In [8]:
# Select data to Train/Validate

# Select 80% of the data to train
n = int(0.8*len(data))
train_data = data[:n]
# Validate the model with the remaining 20%
val_data = data[n:]


In [10]:
# Train the model

block_size = 8

# Break the data in blocks
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when inpuyt is tensor([75]) target is tensor(0)
when inpuyt is tensor([75,  0]) target is tensor(0)
when inpuyt is tensor([75,  0,  0]) target is tensor(27)
when inpuyt is tensor([75,  0,  0, 27]) target is tensor(63)
when inpuyt is tensor([75,  0,  0, 27, 63]) target is tensor(66)
when inpuyt is tensor([75,  0,  0, 27, 63, 66]) target is tensor(63)
when inpuyt is tensor([75,  0,  0, 27, 63, 66, 63]) target is tensor(68)
when inpuyt is tensor([75,  0,  0, 27, 63, 66, 63, 68]) target is tensor(56)
